In [ ]:
import diagonal_b6 as b6

# !conda install pandas -y
# !conda install matplotlib -y

import pandas as pd

In [ ]:
# connect to the world
grpc_address = "localhost:8002"
w = b6.connect_insecure(grpc_address)

In [ ]:
OAKLEY_SQUARE_GARDENS_WAY_ID = 26717682
OAKLEY_SQUARE_GARDENS_BENCH_NODE_ID = 2345394864

## The world?
One of the huge benefits of bedrock is that it allows you to build up a query or analysis without evaluating it. 
You evaluate your query, for example, via applying `w` to it. 
You will see this across the examples as `w(query)`

## Tags

`b6` lets you search the world of osm in whatever data you have loaded. 
You do this via OSM tags (https://wiki.openstreetmap.org/wiki/Tags). Not all tags are "searchable", the ones that are have a `#` up front. Here are ones we have made searchable by default:

	"amenity":   "#amenity"
	"barrier":   "#barrier"
	"boundary":  "#boundary"
	"bridge":    "#bridge"
	"building":  "#building"
	"highway":   "#highway"
	"landuse":   "#landuse"
	"leisure":   "#leisure"
	"natural":   "#natural"
	"network":   "#network"
	"place":     "#place"
	"railway":   "#railway"
	"route":     "#route"
	"shop":      "#shop"
	"water":     "#water"
	"waterway":  "#waterway"
	"fhrs:id":   "@fhrs:id"
	"wikidata":  "@wikidata"
	"wikipedia": "@wikipedia"

In [ ]:
# find all the parks - https://wiki.openstreetmap.org/wiki/Tag:leisure%3Dpark
parks = b6.find( b6.tagged("#leisure","park") )

# access the metadata via `all_tags()`
for osmid,park in w(parks):
    print( f"{osmid}\n{park.all_tags()}" )

## OSM IDs
`b6` is able to find features directly from their `OSM ID`

In [ ]:
# or if you knew the OSM ID of a park you could find it that way
oakleysq_gardens = b6.find_area( b6.osm_way_area_id(OAKLEY_SQUARE_GARDENS_WAY_ID) )

print( w(oakleysq_gardens).all_tags() )

In [ ]:
# you can also search for a bench in the park
bench = b6.find_point( b6.osm_node_id(OAKLEY_SQUARE_GARDENS_BENCH_NODE_ID ))

print( w(bench).all_tags() )

Using the OSM ID you also have access to `b6.osm_way_id` for finding a ways (areas and lines), and for finding relations you have `b6.osm_relation_area_id`, `b6.osm_relation_id`

For search you can always use `b6.find` but if you are being specific, aside from `b6.find_point` and `b6.find_area` there is also `b6.find_path`. More on this below. 


In [ ]:
# If you didn't know what other features were in the park, 
# you could limit your search to what is in the park area
contained_in_oakleysq_gardens = b6.find( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, feature in w(contained_in_oakleysq_gardens):
    print( f"{osmid}\n{feature.all_tags()}" )

---
Note that searching for what is in the park yields different types of features. 

- It yields `nodes`, which in this instance are benches. 
- It yields `ways`, which `b6` splits into `paths` and `areas`
    - `areas` are ways which represent areas, and the rest are `paths`       
- It also yields `access-path` which is also a `path` - these are paths we have automatically generated to ensure all features are connected to the network. This is helpful, for example if you want to compute routes.

In [ ]:
# you could be more precise and only search for `points` within the park
points_in_oakleysq_gardens = b6.find_points( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, point in w(points_in_oakleysq_gardens):
    print( f"{osmid}\n{point.all_tags()}" )

In [ ]:
# you could be more precise and only search for `areas` within the park
areas_in_oakleysq_gardens = b6.find_areas( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, area in w(areas_in_oakleysq_gardens):
    print( f"{osmid}\n{area.all_tags()}" )

In [ ]:
# you could be more precise and only search for `paths` within the park
paths_in_oakleysq_gardens = b6.find_paths( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, path in w(paths_in_oakleysq_gardens):
    print( f"{osmid}\n{path.all_tags()}" )

##  Filter

You can use `.filter` to narrow down your search.
You can also then apply `.map` to grab the value of a tag.
When grabbing the value of a tag you have four options:
- `get` which gets you whatever the value is
- `get_string`
- `get_int`
- `get_float`

In [ ]:
# you can search for features and filter to only get features that contain a certain tag
contained_in_oakleysq_gardens = b6.find( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )
amenities_in_oakleysq_gardens = contained_in_oakleysq_gardens.filter(lambda a: b6.has_key(a, "#amenity"))

for osmid, amenity in w(amenities_in_oakleysq_gardens):
    print( f"{osmid}\n{amenity.all_tags()}" )

In [ ]:
# more generally you can do a wide search, using the searchable tags,
# then do a filter for features that have other non-searchable tags, present. 
offices_with_levels = b6.find_areas( b6.tagged("#building","office") ) \
                          .filter(lambda b: b6.has_key(b, "building:levels"))

for osmid, office in w(offices_with_levels):
    print( f"{osmid}\n{office.all_tags()}" )

In [ ]:
# even more fun, you can extract the value of that tag
levels_in_offices = offices_with_levels.map(lambda b: b6.get_int(b, "building:levels"))

for osmid, levels in w(levels_in_offices):
    print( f"{osmid} -->{levels} levels" )

In [ ]:
# you can use that to then analyse/categorise/filter by value as you need, using python
df = pd.DataFrame(w(levels_in_offices),
                  columns =["office","number_of_levels"]
                 ).set_index("office")

df.plot.hist()